### Word Videos Select

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd
import glob
from gtts import gTTS
from PIL import Image, ImageFilter, ImageOps
#from playsound import playsound
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips,CompositeVideoClip, CompositeAudioClip
import pandas as pd
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import cv2

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [3]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

#### Output Path Folder File

In [4]:
# Output path and folder ???????
result_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words"
main_result_folder = "Deployment3 Select"  # Fourgram_Selected, Test2
result_folder = f"Word Video Select"  # Twogram, Threegram, Fourgram, Sample


In [5]:
# ///***
output_path = f"{result_path}/{main_result_folder}/{result_folder}"  # output path

Path(output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

#### Read Data

In [6]:
# image video path folder
group_num = 1 # 1,2,3
# word
word_video_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Word{group_num}/Result/Image_Video_Audio"

In [7]:
df_deployment_word_master_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/\
2-1-Word Prefix Suffix Analysis/{lang_folder.capitalize()}_1000_Word_Person_Case_Tense_Plural_Mean_Manuel.xlsx", sheet_name="Sheet1")
df_deployment_word_master_file.drop_duplicates(inplace=True)
df_deployment_word_master_file.reset_index(drop=True, inplace=True)
df_deployment_word_master_file

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735,+,NaN,NaN,NaN,NaN,NaN,NaN
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az,NaN,NaN,NaN,NaN,NaN,NaN
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i,NaN,NaN,belirtme,NaN,NaN,NaN
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte,NaN,NaN,NaN,NaN,NaN,NaN
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini,NaN,NaN,belirtme,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um,NaN,1.tekil,NaN,NaN,NaN,NaN
1620,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+,şimdiki,NaN,NaN,NaN,NaN,NaN
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+,NaN,NaN,NaN,NaN,NaN,NaN
1622,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+,NaN,3.tekil,belirtme,NaN,NaN,NaN


In [8]:
df_deployment_ngram_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/Manuel Select/\
Revised Ngrams All in One.xlsx", sheet_name="Sheet2")
df_deployment_ngram_file.drop_duplicates(inplace=True)
df_deployment_ngram_file.reset_index(drop=True, inplace=True)
df_deployment_ngram_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,yardım eder misin,eder misin
2,bana yardım eder misin,yardım eder misin,yardım eder
3,başka bir şey daha,başka bir şey,bir şey
4,başka bir şey daha,başka bir şey,başka bir
...,...,...,...
475,NaN,çok kötü bir,çok kötü
476,NaN,bir şey olmaz,NaN
477,NaN,şuna bir bak,bir bak
478,NaN,şuna bir bak,NaN


In [9]:
df_word_used_ngram_count = word_count_result(df_deployment_ngram_file, ["fourgram","threegram","twogram"], set_condition=True)
df_word_used_ngram_count

,word,word_count
0,bir,124
1,şey,90
2,ne,72
3,var,46
4,bu,36
...,...,...
155,peki,1
156,dur,1
157,pek,1
158,kimsin,1


In [10]:
used_word_list = list(df_word_used_ngram_count["word"])
#used_word_list

In [11]:
# if you want remove previous used word use this
#df_deployment_word_remain = df_deployment_word_master_file[~df_deployment_word_master_file["word"].isin(used_word_list)]
#df_deployment_word_remain

In [12]:
df_deployment_word_remain = df_deployment_word_master_file
df_deployment_word_remain

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
0,bir,NUM,NaN,bir,bir,bir,a,a,18835735,+,NaN,NaN,NaN,NaN,NaN,NaN
1,bir,ADV,NaN,biraz,biraz,bir,a little,a little,1269641,+az,NaN,NaN,NaN,NaN,NaN,NaN
2,bir,PRON,NaN,biri,biri,bir,somebody,somebody,837400,+i,NaN,NaN,belirtme,NaN,NaN,NaN
3,bir,ADV,NaN,birlikte,birlikte,bir,together,together,409940,+likte,NaN,NaN,NaN,NaN,NaN,NaN
4,bir,NOUN,NaN,birini,biri,bir,one,somebody,259916,+ini,NaN,NaN,belirtme,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,veriyor,VERB,NaN,veriyorum,ver,ver,i give,give,126290,+um,NaN,1.tekil,NaN,NaN,NaN,NaN
1620,veriyor,VERB,NaN,veriyor,ver,ver,giving,give,68163,+,şimdiki,NaN,NaN,NaN,NaN,NaN
1621,okul,NOUN,NaN,okul,okul,oku,school,school,68160,+,NaN,NaN,NaN,NaN,NaN,NaN
1622,suçlu,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124,+,NaN,3.tekil,belirtme,NaN,NaN,NaN


#### Select Word Videos

##### Tenses

###### Past

In [13]:
# ///***
add_past_out_folder = "Geçmiş"

past_output_path = f"{output_path}/{add_past_out_folder}"

Path(past_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [14]:
df_past_tense_result = df_deployment_word_remain[df_deployment_word_remain["tense"] == "geçmiş"]
df_past_tense_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
57,de,VERB,NaN,dedim,de,de,i said,say,267039,+dim,geçmiş,1.tekil,NaN,NaN,NaN,NaN
58,de,VERB,NaN,dedi,de,de,said,also,182523,+di,geçmiş,3.tekil,NaN,NaN,NaN,NaN
60,de,VERB,NaN,dedin,de,de,you said,also,121622,+din,geçmiş,2.tekil,NaN,NaN,NaN,NaN
66,var,VERB,NaN,vardı,var,var,there was,there is,451341,+dı,geçmiş,NaN,NaN,NaN,NaN,to be
69,var,VERB,NaN,varmış,var,var,there was,there is,118304,+mış,geçmiş,NaN,NaN,NaN,NaN,to be
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,yapmış,VERB,NaN,yapmış,yap,yap,made,do it,73719,+,geçmiş,3.tekil,NaN,NaN,NaN,NaN
1600,istemedim,VERB,NEG,istemedim,iste,iste,i didn't want,want,68907,+,geçmiş,1.tekil,NaN,NaN,olumsuz,NaN
1610,bul,VERB,NaN,buldum,bul,bul,i found,find,179420,+dum,geçmiş,1.tekil,NaN,NaN,NaN,NaN
1612,bul,VERB,NaN,buldun,bul,bul,you found,find,91964,+dun,geçmiş,2.tekil,NaN,NaN,NaN,NaN


In [15]:
past_word_list = list(set(df_past_tense_result["word"]))
#past_word_list

In [16]:
for past_word in past_word_list:
    source = f"{word_video_path_folder}/{past_word}.mp4"
    destination = past_output_path
    shutil.copy2(source, destination)

###### Present

In [17]:
# ///***
add_present_out_folder = "Şimdiki"

present_output_path = f"{output_path}/{add_present_out_folder}"

Path(present_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [18]:
df_present_tense_result = df_deployment_word_remain[df_deployment_word_remain["tense"] == "şimdiki"]
df_present_tense_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
185,bile,VERB,NEG,bilemiyorum,bil,bil,i do not know,know,82742,+miyorum,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
197,istiyorum,VERB,NaN,istiyorum,iste,iste,i want,want,978284,+,şimdiki,1.tekil,NaN,NaN,NaN,NaN
201,biliyorum,VERB,NaN,biliyorum,bil,bil,i know,know,935083,+,şimdiki,1.tekil,NaN,NaN,NaN,NaN
249,bilmiyorum,VERB,NEG,bilmiyorum,bil,bil,i do not know,know,700046,+,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
281,biliyor,VERB,NaN,biliyorum,bil,bil,i know,know,935083,+um,şimdiki,1.tekil,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,istemiyor,VERB,NEG,istemiyorum,iste,iste,i don't want,want,345472,+um,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
1509,istemiyor,VERB,NEG,istemiyor,iste,iste,does not want,want,75926,+,şimdiki,3.tekil,NaN,NaN,olumsuz,NaN
1553,duyuyor,VERB,NaN,duyuyor,duy,duy,hears,heard,72744,+,şimdiki,3.tekil,NaN,NaN,NaN,NaN
1575,istiyorsunuz,VERB,NaN,istiyorsunuz,iste,iste,you want,want,71208,+,şimdiki,2.çoğul,NaN,NaN,NaN,NaN


In [19]:
present_word_list = list(set(df_present_tense_result["word"]))
#present_word_list

In [20]:
for present_word in present_word_list:
    source = f"{word_video_path_folder}/{present_word}.mp4"
    destination = present_output_path
    shutil.copy2(source, destination)

###### Future

In [21]:
# ///***
add_future_out_folder = "Gelecek"

future_output_path = f"{output_path}/{add_future_out_folder}"

Path(future_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [22]:
df_future_tense_result = df_deployment_word_remain[df_deployment_word_remain["tense"] == "gelecek"]
df_future_tense_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
227,olacak,VERB,NaN,olacak,ol,ol,will be,be,786194,+,gelecek,1.çoğul,NaN,NaN,NaN,to be
228,olacak,VERB,NaN,olacaksın,ol,ol,you will be,die,95544,+sın,gelecek,2.tekil,NaN,NaN,NaN,to be
305,ol,VERB,NaN,olacak,ol,ol,will be,be,786194,+acak,gelecek,NaN,NaN,NaN,NaN,to be
324,ol,VERB,NaN,olacağım,ol,ol,i will be,be,150030,+acağım,gelecek,1.tekil,NaN,NaN,NaN,to be
328,ol,VERB,NEG,olmayacak,ol,ol,will not be,be,132589,+mayacak,gelecek,3.tekil,NaN,NaN,olumsuz,to be
333,ol,VERB,NaN,olacaksın,ol,ol,you will be,die,95544,+acaksın,gelecek,2.tekil,NaN,NaN,NaN,to be
383,ver,VERB,NaN,vereceğim,ver,ver,i will give,give,73483,+eceğim,gelecek,1.tekil,NaN,NaN,NaN,NaN
483,al,VERB,NaN,alacağım,al,al,i will take,get,82272,+acağım,gelecek,1.tekil,NaN,NaN,NaN,NaN
758,yap,VERB,NaN,yapacağım,yap,yap,i will do it,do it,135540,+acağım,gelecek,1.tekil,NaN,NaN,NaN,NaN
760,yap,VERB,NaN,yapacağız,yap,yap,we will do,do it,120078,+acağız,gelecek,1.çoğul,NaN,NaN,NaN,NaN


In [23]:
future_word_list = list(set(df_future_tense_result["word"]))
#future_word_list

In [24]:
for future_word in future_word_list:
    source = f"{word_video_path_folder}/{future_word}.mp4"
    destination = future_output_path
    shutil.copy2(source, destination)

##### Negative

In [25]:
# ///***
add_negative_out_folder = "Olumsuz"

negative_output_path = f"{output_path}/{add_negative_out_folder}"

Path(negative_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [26]:
df_negative_mean_result = df_deployment_word_remain[df_deployment_word_remain["mean"] == "olumsuz"]
df_negative_mean_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
75,değil,CCONJ,NEG,değil,değil,değil,not,not,3883885,+,NaN,NaN,NaN,NaN,olumsuz,NaN
76,değil,VERB,NEG,değilim,değil,değil,i am not,not,448632,+im,NaN,1.tekil,NaN,NaN,olumsuz,NaN
77,değil,VERB,NEG,değildi,değil,değil,it wasn't,not,235756,+di,geçmiş,NaN,NaN,NaN,olumsuz,NaN
78,değil,VERB,NEG,değilsin,değil,değil,you are not,not,215503,+sin,NaN,2.tekil,NaN,NaN,olumsuz,NaN
79,değil,VERB,NEG,değildir,değil,değil,it is not,not,126096,+dir,NaN,NaN,NaN,NaN,olumsuz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,anlamı,VERB,NEG,anlamıyorum,anla,anla,i don't understand,understand,91669,+yorum,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
1508,istemiyor,VERB,NEG,istemiyorum,iste,iste,i don't want,want,345472,+um,şimdiki,1.tekil,NaN,NaN,olumsuz,NaN
1509,istemiyor,VERB,NEG,istemiyor,iste,iste,does not want,want,75926,+,şimdiki,3.tekil,NaN,NaN,olumsuz,NaN
1518,görmedim,VERB,NEG,görmedim,gör,gör,i did not see,see,75297,+,geçmiş,1.tekil,NaN,NaN,olumsuz,NaN


In [27]:
negative_word_list = list(set(df_negative_mean_result["word"]))
#negative_word_list

In [28]:
for negative_word in negative_word_list:
    source = f"{word_video_path_folder}/{negative_word}.mp4"
    destination = negative_output_path
    shutil.copy2(source, destination)

##### Imperative

In [29]:
# ///***
add_imperative_out_folder = "Emir"

imperative_output_path = f"{output_path}/{add_imperative_out_folder}"

Path(imperative_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [30]:
df_imperative_word_result = df_deployment_word_remain[df_deployment_word_remain["other"] == "imperative"]
df_imperative_word_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
191,bak,VERB,NaN,bak,bak,bak,look,look,1016510,+,NaN,NaN,NaN,NaN,NaN,imperative
193,bak,VERB,NaN,bakın,bak,bak,look,look,256830,+ın,NaN,2.çoğul,NaN,NaN,NaN,imperative
194,bak,VERB,NEG,bakma,bak,bak,do not look,look,100165,+ma,NaN,NaN,NaN,NaN,olumsuz,imperative
195,bak,VERB,NaN,baksana,baksana,bak,look,look,85248,+sana,NaN,NaN,NaN,NaN,NaN,imperative
267,et,VERB,NaN,et,et,et,do,do,673946,+,NaN,NaN,NaN,NaN,NaN,imperative
270,et,VERB,NEG,etme,et,et,don't,do,218533,+me,NaN,NaN,NaN,NaN,olumsuz,imperative
355,gel,VERB,NaN,gel,gel,gel,come,come,561929,+,NaN,NaN,NaN,NaN,NaN,imperative
376,ver,VERB,NaN,ver,ver,ver,give,give,531352,+,NaN,2.tekil,NaN,NaN,NaN,imperative
378,ver,VERB,NaN,verin,ver,ver,give,give,139708,+in,NaN,2.tekil,NaN,NaN,NaN,imperative
418,dur,AUX,NaN,dur,dur,dur,is/are,is/are,483312,+,NaN,NaN,NaN,NaN,NaN,imperative


In [31]:
imperative_word_list = list(set(df_imperative_word_result["word"]))
#imperative_word_list

In [32]:
for imperative_word in imperative_word_list:
    source = f"{word_video_path_folder}/{imperative_word}.mp4"
    destination = imperative_output_path
    shutil.copy2(source, destination)

##### Question

In [33]:
# ///***
add_question_out_folder = "Soru"

question_output_path = f"{output_path}/{add_question_out_folder}"

Path(question_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [34]:
df_question_word_result = df_deployment_word_remain[df_deployment_word_remain["POS2"] == "Q"]
df_question_word_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
19,ne,PRON,Q,ne,ne,ne,what,what,8025880,+,NaN,NaN,NaN,NaN,NaN,NaN
20,ne,NOUN,Q,neler,ne,ne,what,what,397377,+ler,NaN,NaN,NaN,çoğul,NaN,NaN
21,ne,NOUN,Q,nedir,ne,ne,what,what,378042,+dir,NaN,NaN,NaN,NaN,NaN,NaN
22,ne,NOUN,Q,neyi,ne,ne,what,what,124260,+yi,NaN,3.tekil,belirtme,NaN,NaN,NaN
23,ne,NOUN,Q,neydi,ne,ne,what was it,what,123671,+ydi,NaN,NaN,NaN,NaN,NaN,NaN
24,ne,NOUN,Q,neyin,ne,ne,what,what,119777,+yin,NaN,2.tekil,NaN,NaN,NaN,NaN
25,ne,NOUN,Q,neye,ne,ne,to what,what,98750,+ye,NaN,NaN,yönelme,NaN,NaN,NaN
129,neden,NOUN,Q,neden,neden,neden,why,why,1866089,+,NaN,NaN,NaN,NaN,NaN,NaN
142,nasıl,ADV,Q,nasıl,nasıl,nasıl,how,how,1656739,+,NaN,NaN,NaN,NaN,NaN,NaN
143,nasıl,NOUN,Q,nasılsın,nasıl,nasıl,how are you,how,130778,+sın,NaN,2.tekil,NaN,NaN,NaN,NaN


In [35]:
question_word_list = list(set(df_question_word_result["word"]))
#question_word_list

In [36]:
for question_word in question_word_list:
    source = f"{word_video_path_folder}/{question_word}.mp4"
    destination = question_output_path
    shutil.copy2(source, destination)

##### Plural

In [37]:
# ///***
add_plural_out_folder = "Çoğul"

plural_output_path = f"{output_path}/{add_plural_out_folder}"

Path(plural_output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [38]:
df_plural_word_result = df_deployment_word_remain[df_deployment_word_remain["plural"] == "çoğul"]
df_plural_word_result

,search_word,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,prefix_suffix,tense,person,case,plural,mean,other
14,bu,PRON,NaN,bunlar,bu,bu,these,this,315388,+nlar,NaN,NaN,NaN,çoğul,NaN,NaN
16,bu,PRON,NaN,bunları,bu,bu,these,this,204443,+nları,NaN,NaN,belirtme,çoğul,NaN,NaN
18,bu,PRON,NaN,bunların,bu,bu,these,this,99699,+nların,NaN,NaN,NaN,çoğul,NaN,NaN
20,ne,NOUN,Q,neler,ne,ne,what,what,397377,+ler,NaN,NaN,NaN,çoğul,NaN,NaN
40,o,PRON,NaN,onları,onlar,o,them,they,575868,+nları,NaN,NaN,belirtme,çoğul,NaN,NaN
41,o,PRON,NaN,onlar,onlar,o,them,he,406681,+nlar,NaN,NaN,NaN,çoğul,NaN,NaN
43,o,PRON,NaN,onlara,onlar,o,to them,he,292433,+nlara,NaN,NaN,yönelme,çoğul,NaN,NaN
45,o,PRON,NaN,onların,onlar,o,their,they,179731,+nların,NaN,NaN,NaN,çoğul,NaN,NaN
46,o,NOUN,NaN,onlardan,onlar,o,from them,they,82961,+nlardan,NaN,NaN,ayrılma,çoğul,NaN,NaN
47,o,PRON,NaN,onlarla,onlar,o,with them,they,79733,+nlarla,NaN,NaN,NaN,çoğul,NaN,NaN


In [39]:
plural_word_list = list(set(df_plural_word_result["word"]))
#plural_word_list

In [40]:
for plural_word in plural_word_list:
    source = f"{word_video_path_folder}/{plural_word}.mp4"
    destination = plural_output_path
    shutil.copy2(source, destination)

#### Result Video File Concat

In [54]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import nltk
from nltk import word_tokenize

In [55]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

filename_length_num = 5

In [56]:
def file_path_concat_video(video_file_list, out_folder_name, out_path, filename):  # will test
    '''
    file_path_concat_video(video_file_list, out_folder_name, out_path, filename)\n
    video_file_list is list of videos and it is with file path. out_folder_name and out_path are path and folder in string(str)\n
    filename is string of result mp4 file name.
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/200 Words"\n
    file_path_concat_video(data_file_list, "Concat_Video", path, "200_Word_Combined_Sample")
    '''
    no_image_file_list = []
    result_path = f"{out_path}/{out_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    video_text_list = []
    video_list = []

    if isfile(f"{result_path}/{filename}.mp4"):
        pass
    else:
        for video_file in video_file_list:
            if isfile(f"{video_file}"):
                video_text_list.append(video_file)            
            else:
                no_image_file_list.append(video_file)

        for video_num in range(len(video_text_list)):
            try:
                clip_var = VideoFileClip(f"{video_text_list[video_num]}", target_resolution=(900, 1600))  # height, width
                video_list.append(clip_var)            
            except:
                print("Oops!", sys.exc_info()[0], "occurred.", f"{video_text_list[video_num]}")

        combined = concatenate_videoclips(video_list, method='compose')
        combined.write_videofile(f"{result_path}/{filename}.mp4", audio_bitrate='128k', audio_fps=44100, audio_codec="aac")
    
    return no_image_file_list

##### Data And Result Path Folder

In [57]:
# Data path folder (previous part Result(Output) path and folder)
result_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words"
main_result_folder = "Deployment3 Select" 
result_folder = f"Word Video Select"

In [58]:
# Output path folder
prev_result_path = f"{result_path}"
prev_main_result_folder = f"{main_result_folder}"
prev_result_folder = f"{result_folder}"
add_data_folder = "Şimdiki"  # Geçmiş, Şimdiki, Gelecek, Olumsuz, Emir, Çoğul, Soru

In [59]:
data_path = f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}"  # previous output path result for data for this
result_path = f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}/{add_data_folder}"

In [60]:
result_path

'/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki'

##### Read Data And Result

In [61]:
#data_file_list = glob.glob(f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}/{add_data_folder}/*.mp4")
data_file_list = glob.glob(f"{data_path}/{add_data_folder}/*.mp4")
data_file_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/diyorsun.mp4',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/çalışıyorum.mp4',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/sanmıyorum.mp4',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/istiyorsun.mp4',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/görüyorum.mp4',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/bekliyor.mp4',
 '

In [62]:
file_path_concat_video(data_file_list, "All_Concat_Video", result_path, f"{add_data_folder}")

Moviepy - Building video /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/All_Concat_Video/Şimdiki.mp4.
MoviePy - Writing audio in ŞimdikiTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/All_Concat_Video/Şimdiki.mp4



Moviepy - Done !
Moviepy - video ready /media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words/Deployment3 Select/Word Video Select/Şimdiki/All_Concat_Video/Şimdiki.mp4


[]